# GPU 확인

In [ ]:
!nvidia-smi

# google drive mount

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive/')

# setup
setting TF2 Object Detection API

In [ ]:
# Clone the tensorflow models repository
!git clone --depth 1 https://github.com/tensorflow/models

In [ ]:
%cd models/research
# Compile protos.
!protoc object_detection/protos/*.proto --python_out=.
# Install TensorFlow Object Detection API.
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .

### setting test

In [ ]:
!python object_detection/builders/model_builder_tf2_test.py

# Move workspace

In [ ]:
!cp -r /content/gdrive/MyDrive/RUP/SSD/* /content

# tensorboard
train tensorboard

In [ ]:
%cd /content/workspace/training_demo
%load_ext tensorboard
%tensorboard --logdir 'models/my_ssd_resnet50_v1_fpn'

# training

### train code copy

In [ ]:
!cp /content/models/research/object_detection/model_main_tf2.py /content/workspace/training_demo/

### train start

In [ ]:
%cd /content/workspace/training_demo
!python model_main_tf2.py --model_dir=models/my_ssd_resnet50_v1_fpn --pipeline_config_path=models/my_ssd_resnet50_v1_fpn/pipeline.config

# Evaluating the Model

### tensorboard

In [ ]:
%cd /content/workspace/training_demo
%load_ext tensorboard
%tensorboard --logdir 'exported-models/RUP/eval'

### evaluating

In [ ]:
%cd /content/workspace/training_demo
!python model_main_tf2.py --model_dir=exported-models/RUPv2 --pipeline_config_path=exported-models/RUPv2/pipeline.config --checkpoint_dir=exported-models/RUPv2/checkpoint

# Exporting a Trained Model

### copy py

In [ ]:
!cp /content/models/research/object_detection/exporter_main_v2.py /content/workspace/training_demo/

### export start

In [ ]:
%cd /content/workspace/training_demo
!python exporter_main_v2.py --input_type image_tensor --pipeline_config_path models/my_ssd_resnet50_v1_fpn/pipeline.config --trained_checkpoint_dir models/my_ssd_resnet50_v1_fpn/ --output_directory exported-models/RUPv2

### move files

In [ ]:
!cp -r /content/workspace/training_demo/exported-models/RUPv2 /content/gdrive/MyDrive/RUP/SSD/workspace/training_demo/exported-models/

# predict

### import

In [ ]:
import io
import os
import scipy.misc
import numpy as np
import six
import time

from six import BytesIO

import matplotlib
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont
from glob import glob

import tensorflow as tf
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import label_map_util

%matplotlib inline

### load model
path = '/content/workspace/training_demo/exported-models/RUPv2'<br>
model name = 'detect_fn'

In [ ]:
PATH_TO_MODEL_DIR = '/content/workspace/training_demo/exported-models/RUPv2'

PATH_TO_SAVED_MODEL = PATH_TO_MODEL_DIR + "/saved_model"

print('Loading model...', end='')
start_time = time.time()

# Load saved model and build the detection function
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

### predict

In [ ]:
%cd /content

In [ ]:
def load_image_into_numpy_array(path):
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [ ]:
category_index = {
    1: {'id': 1, 'name': 'pet'},
    2: {'id': 2, 'name': 'ps'},
}

In [ ]:
image_dir = glob('/content/workspace/training_demo/predict/*.png')

elapsed = []
for image in image_dir:
  image_np = load_image_into_numpy_array(image)
  input_tensor = np.expand_dims(image_np, 0)
  start_time = time.time()
  detections = detect_fn(input_tensor)
  end_time = time.time()
  elapsed.append(end_time - start_time)

  plt.rcParams['figure.figsize'] = [42, 21]
  label_id_offset = 1
  image_np_with_detections = image_np.copy()
  viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_detections,
        detections['detection_boxes'][0].numpy(),
        detections['detection_classes'][0].numpy().astype(np.int32),    #label 종류 리스트
        detections['detection_scores'][0].numpy(),                      #label 별로 정확도 리스트
        category_index,
        use_normalized_coordinates=True,
        max_boxes_to_draw=200,
        min_score_thresh=.40,
        agnostic_mode=False)
  plt.imshow(image_np_with_detections)
  plt.show()

mean_elapsed = sum(elapsed) / float(len(elapsed))
print('Elapsed time: ' + str(mean_elapsed) + ' second per image')